## **Model Inference**
Pada tahap ini, saya akan menggunakan model dan komponen preprocessing yang sudah disimpan pada tahap sebelumnya untuk membuat prediksi nasabah baru ini masuk ke dalam segmentasi/cluster mana. 

In [58]:
# Import Libraries
import pickle
import pandas as pd
import numpy as np

# Mengabaikan warning
import warnings
warnings.filterwarnings('ignore')

**1. Load model**


In [63]:
with open('kproto_bundle.pkl', 'rb') as f:
    model_bundle = pickle.load(f)

kproto = model_bundle['kproto_model']
pca = model_bundle['pca_model']
scaler = model_bundle['scaler']
numerical_cols = model_bundle['numerical_cols']
categorical_cols = model_bundle['categorical_cols']
final_columns_order = model_bundle['final_columns_order']
categorical_indices = model_bundle['categorical_indices']  

**2. Mengisi data user baru**

In [66]:
# Pastikan semua kolom yang ada di komponen preprocessing ada di sini dengan nama yang sama persis seperti saat training.
# Data user baru
new_user_raw = pd.DataFrame([{
        'call_failure': 10, # Pastikan nama kolom ini sama persis dengan saat training
        'subscription_length': 37,
        'seconds_of_use': 2453.0,
        'frequency_of_use': 60,
        'frequency_of_sms': 209.5,
        'distinct_called_numbers': 24,
        'customer_value': 1536.52,
        'charge_amount': 0,
        'age_group': 3,
        'complains':0,
        'tariff_plan': 1,
        'status': 1,
        'age':30,
}])
new_user = new_user_raw[final_columns_order]
new_user

,complains,subscription_length,charge_amount,seconds_of_use,frequency_of_use,distinct_called_numbers,age_group,tariff_plan,status,customer_value
0,0,37,0,2453.0,60,24,3,1,1,1536.52


**3. Preprocessing & Predict untuk Inference**

In [ ]:
# Preprocessing
numerical_data = new_user[numerical_cols]
categorical_data = new_user[categorical_cols].astype(str)

# Transform
num_scaled = scaler.transform(numerical_data)
num_pca = pca.transform(num_scaled)

# Gabungkan
final_input = np.hstack([num_pca, categorical_data.to_numpy()])

# Debug shape dan tipe (sementara aja)
print("Final input shape:", final_input.shape)
print("Categorical indices:", categorical_indices)

# Predict
predicted_cluster = kproto.predict(final_input, categorical=categorical_indices)
print(f"Data pelanggan diprediksi masuk ke dalam Klaster: {predicted_cluster[0]}")

# Tambahkan ke dataframe
new_user['cluster'] = predicted_cluster
new_user.T

Final input shape: (1, 9)
Categorical indices: [6, 7, 8]
Data pelanggan diprediksi masuk ke dalam Klaster: 0


,0
complains,0.00
subscription_length,37.00
charge_amount,0.00
seconds_of_use,2453.00
frequency_of_use,60.00
distinct_called_numbers,24.00
age_group,3.00
tariff_plan,1.00
status,1.00
customer_value,1536.52
